In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
import peakutils
from sklearn.linear_model import LogisticRegression

#from mpl_toolkits.mplot3d import Axes3D

sensorData = []
soundData  = []
labels = []

punch0 = []
punch1 = []
punch2 = []
punch3 = []
punch4 = []


 
with open ('sensors3.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        line = []
        for x in row.split(','):
            line1 = []
            for y in x.split(' '):
                try:
                    line1.append(float(y))
                except Exception:
                    pass
            if len(line1) == 6:
                line.append(line1)
        if len(line) == 250:
            sensorData.append(line)
with open ('sounds3.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        empty = []
        for i in row.split(','):
            empty.append(float(i))
        soundData.append(empty)
with open ('labels3.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        labels.append(int(row))

for x in range(len(sensorData)):
    if x % 5 == 0:
        punch0.append(sensorData[x])
    if x % 5 == 1:
        punch1.append(sensorData[x])
    if x % 5 == 2:
        punch2.append(sensorData[x])
    if x % 5 == 3:
        punch3.append(sensorData[x])
    if x % 5 == 4:
        punch4.append(sensorData[x])
soundData  = np.array(soundData)
sensorData = np.array(sensorData)
labels     = np.array(labels)
punch0 = np.array(punch0)
punch1 = np.array(punch1)
punch2 = np.array(punch2)
punch3 = np.array(punch3)
punch4 = np.array(punch4)

print(soundData.shape)
print(sensorData.shape)


(600, 5000)
(600, 250, 6)


In [2]:
from scipy.fftpack import fft, rfft
sensorData = np.array(sensorData)

def zeroCrossings(x):
    zero_crossings = np.where(np.diff(np.sign(x)))[0]
    return(len(zero_crossings))
    

def featureExtraction(x, sound):
    features = []
    for u in range(6):
        features.append(np.mean(x[:100, u]))
        features.append(np.max(x[:100, u]))
        features.append(np.min(x[:100, u]))
        features.append(np.std(x[:100, u]))
        features.append(zeroCrossings(x[:100, u]))
        features.append(np.var(x[:100, u]))
    features.append(np.std(x[:100, :3]))
    features.append(np.std(x[:100, 3:]))
    a, b, c, d, e, f, g, h, i = topPeaks(sound)
    features.append(a)
    features.append(b)
    features.append(c)
    features.append(d)
    features.append(e)
    features.append(f)
    features.append(g)
    features.append(h)
    features.append(i)
    return features

def topPeaks(sound):
    fourier = abs(rfft(sound))
    phase = (np.angle(rfft(sound)))
    indices = peakutils.indexes(fourier, thres=0.02/max(fourier), min_dist=0.1)
    amplitudes = (fourier[indices])
    amplitudes = (np.sort(amplitudes))[len(amplitudes)-3:len(amplitudes)]
    a    = fourier.tolist().index(amplitudes[0])
    b    = fourier.tolist().index(amplitudes[1])
    c    = fourier.tolist().index(amplitudes[2])
    d, e, f = amplitudes
    g    = phase[a]
    h    = phase[b]
    i    = phase[c]
    return a, b, c, d, e, f, g, h, i

topPeaks(soundData[250,0:])
# for x in range(len(sensorData)):
#     print(x)
#     print(featureExtraction(sensorData[x], soundData[x, 0:]))  
features = []

for x in range(len(sensorData)):
     features.append(featureExtraction(sensorData[x], soundData[x, 0:]))

features = np.array(features)

training = features[:500]
test     = features[500:550]
validation = features[550:]
training_labels = labels[:500]
test_labels = labels[500:550]
validaton_labels = labels[550:]

print(soundData.shape)
print(training.shape)
print(test.shape)

def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation]
    shuffled_labels  = labels[permutation]
    return shuffled_dataset, shuffled_labels

full_dataset, full_labels = randomize(features, labels)
train_dataset, train_labels = randomize(training, training_labels)
test_dataset, test_labels = randomize(test, test_labels)
validation_dataset, validation_labels = randomize(validation, validaton_labels)



(600, 5000)
(500, 47)
(50, 47)


In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from tensorflow.contrib import learn
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from numpy import reciprocal

In [ ]:
# params = {
#     'kernel': ['poly', 'linear', 'rbf', 'sigmoid'],
# }

# clf = SVC()
# rnd_search = GridSearchCV(clf, param_grid=params)
# rnd_search.fit(train_dataset,train_labels)


In [ ]:
# rnd_search.best_params_

In [ ]:
# classifier = LinearSVC(random_state=42, penalty='l1', max_iter=40000)
# classifier.fit(train_dataset, train_labels)

In [ ]:
# classifier.score(test_dataset, test_labels)

In [ ]:
# params1 = {
#     'hidden_layer_sizes': [(27,3), (28,3), (29,3), (27, 2), (28, 2), (29, 2)],
#     'activation': ['logistic', 'tanh', 'relu'],
#     'solver': ['lbfgs', 'sgd'],
#     'max_iter': [5000, 6000, 7000],
#     'random_state': [42],
#     'learning_rate': ['constant', 'adaptive'],
#     'learning_rate_init': [0.1]
# }
# clf1 = MLPClassifier()
# rnd_search1 = GridSearchCV(clf1, param_grid=params1)
# rnd_search1.fit(train_dataset,train_labels)

In [ ]:
# rnd_search1.best_score_
# rnd_search1.best_estimator_


In [ ]:
# classifier1 =MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
#        beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
#        hidden_layer_sizes=(29, 3), learning_rate='constant',
#        learning_rate_init=0.1, max_iter=5000, momentum=0.9,
#        nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
#        solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
#        warm_start=False)
# classifier1.fit(train_dataset, train_labels)
# classifier1.score(train_dataset, train_labels)


In [ ]:
# for i in range(1, 300):
#     classifier1 = MLPClassifier(hidden_layer_sizes= (i, ), solver='sgd', random_state=42, max_iter=100000, learning_rate= 'constant', activation='logistic', learning_rate_init=0.01) 
#     classifier1.fit(train_dataset, train_labels)
#     if classifier1.score(test_dataset, test_labels) > 0.6:
#         print(i)

    


In [12]:

feature_columns = learn.infer_real_valued_columns_from_input(train_dataset)
classifier = learn.LinearClassifier(n_classes = 5, feature_columns=feature_columns,
                                    optimizer=tf.train.FtrlOptimizer(
                                    learning_rate_power=-0.69,
                                    learning_rate=0.00001,
                                    l1_regularization_strength=0.1))
#classifier = learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[10, 20, 10], n_classes=5)
classifier.fit(train_dataset, train_labels, steps=30000)
# classifier.fit(train_dataset, train_labels)





INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4968b4f358>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmp40b1y7_m'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp40b1y7_m/model.ckpt.
INFO:tensorflow:loss = 1.609438, st

INFO:tensorflow:global_step/sec: 745.449
INFO:tensorflow:loss = 0.26474026, step = 7100 (0.134 sec)
INFO:tensorflow:global_step/sec: 742.707
INFO:tensorflow:loss = 0.2638601, step = 7200 (0.135 sec)
INFO:tensorflow:global_step/sec: 736.009
INFO:tensorflow:loss = 0.26299337, step = 7300 (0.136 sec)
INFO:tensorflow:global_step/sec: 729.245
INFO:tensorflow:loss = 0.26213974, step = 7400 (0.137 sec)
INFO:tensorflow:global_step/sec: 739.444
INFO:tensorflow:loss = 0.26129887, step = 7500 (0.135 sec)
INFO:tensorflow:global_step/sec: 746.916
INFO:tensorflow:loss = 0.26047033, step = 7600 (0.134 sec)
INFO:tensorflow:global_step/sec: 737.795
INFO:tensorflow:loss = 0.25965384, step = 7700 (0.135 sec)
INFO:tensorflow:global_step/sec: 735.355
INFO:tensorflow:loss = 0.25884908, step = 7800 (0.136 sec)
INFO:tensorflow:global_step/sec: 740.839
INFO:tensorflow:loss = 0.25805575, step = 7900 (0.135 sec)
INFO:tensorflow:global_step/sec: 627.01
INFO:tensorflow:loss = 0.25727355, step = 8000 (0.160 sec)
IN

INFO:tensorflow:global_step/sec: 714.036
INFO:tensorflow:loss = 0.21864481, step = 15300 (0.139 sec)
INFO:tensorflow:global_step/sec: 726.265
INFO:tensorflow:loss = 0.21827468, step = 15400 (0.138 sec)
INFO:tensorflow:global_step/sec: 662.127
INFO:tensorflow:loss = 0.21790734, step = 15500 (0.151 sec)
INFO:tensorflow:global_step/sec: 679.464
INFO:tensorflow:loss = 0.21754281, step = 15600 (0.147 sec)
INFO:tensorflow:global_step/sec: 712.027
INFO:tensorflow:loss = 0.2171809, step = 15700 (0.140 sec)
INFO:tensorflow:global_step/sec: 518.591
INFO:tensorflow:loss = 0.21682164, step = 15800 (0.193 sec)
INFO:tensorflow:global_step/sec: 696.676
INFO:tensorflow:loss = 0.21646504, step = 15900 (0.144 sec)
INFO:tensorflow:global_step/sec: 706.048
INFO:tensorflow:loss = 0.21611096, step = 16000 (0.142 sec)
INFO:tensorflow:global_step/sec: 651.822
INFO:tensorflow:loss = 0.21575949, step = 16100 (0.154 sec)
INFO:tensorflow:global_step/sec: 616.912
INFO:tensorflow:loss = 0.21541047, step = 16200 (0.

INFO:tensorflow:loss = 0.19528574, step = 23400 (0.167 sec)
INFO:tensorflow:global_step/sec: 610.925
INFO:tensorflow:loss = 0.19505918, step = 23500 (0.164 sec)
INFO:tensorflow:global_step/sec: 591.819
INFO:tensorflow:loss = 0.19483374, step = 23600 (0.169 sec)
INFO:tensorflow:global_step/sec: 600.311
INFO:tensorflow:loss = 0.1946093, step = 23700 (0.167 sec)
INFO:tensorflow:global_step/sec: 588.085
INFO:tensorflow:loss = 0.19438599, step = 23800 (0.170 sec)
INFO:tensorflow:global_step/sec: 632.568
INFO:tensorflow:loss = 0.1941638, step = 23900 (0.158 sec)
INFO:tensorflow:global_step/sec: 639.729
INFO:tensorflow:loss = 0.19394264, step = 24000 (0.156 sec)
INFO:tensorflow:global_step/sec: 622.441
INFO:tensorflow:loss = 0.19372256, step = 24100 (0.162 sec)
INFO:tensorflow:global_step/sec: 563.433
INFO:tensorflow:loss = 0.19350353, step = 24200 (0.176 sec)
INFO:tensorflow:global_step/sec: 578.429
INFO:tensorflow:loss = 0.19328554, step = 24300 (0.173 sec)
INFO:tensorflow:global_step/sec: 

LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f4968b4f438>, 'feature_columns': [_RealValuedColumn(column_name='', dimension=47, default_value=None, dtype=tf.float64, normalizer=None)], 'optimizer': <tensorflow.python.training.ftrl.FtrlOptimizer object at 0x7f4968b4f240>, 'gradient_clip_norm': None, 'joint_weights': False})

In [14]:
classifier.evaluate(validation_dataset, validation_labels)
prediction = []
for i in classifier.predict(test_dataset):
     prediction.append(i)
for i in range(len(prediction)):
    if prediction[i] != test_labels[i]:
        print((prediction[i], test_labels[i]))


INFO:tensorflow:Starting evaluation at 2018-08-14-10:14:18
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp40b1y7_m/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-14-10:14:18
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.86, global_step = 30000, loss = 0.5622111
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp40b1y7_m/model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
(1, 0)
(0, 1)
(3, 4)
(0, 1)
